In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn

In [3]:
OUTPUT_DIR = './tmp/ref_rnn'

In [4]:
get_ipython().system_raw('rm -rf {}'.format(OUTPUT_DIR))

In [5]:
# this will kill the processes for Tensorboard
is_tensorboard = False
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

In [6]:
dataset_type='MNIST'
dataset_type='P7'
print("Dataset type= "+dataset_type)


num_hidden_units = 256#*4*2#//2  # Number of hidden units of the RNN cell
#print(n_classes,batch_size, num_input)
CELL_TYPE = 'LSTM'
CELL_TYPE = 'RNN'
CELL_TYPE = 'SRNN' # For stacked RNN
CELL_TYPE = 'SGRU' # For stacked GRU
CELL_TYPE = 'GRU'

RNN_LAYER=1

learning_rate = 0.001 # The optimization initial learning rate
epoch_number=100# Total number of training epochs ==> train steps = epoch_number*batch_number

if dataset_type == 'MNIST':
    batch_size = 100      # Training batch size
    # Data Dimension
    num_input = 28          # MNIST data input (image shape: 28x28)
    timesteps = 28          # Timesteps
    n_classes = 10          # Number of classes, one class per digit
else :
    batch_size = 69 # 414/6 is 69
    batch_size = 23
    num_input = 224*3      # Dogs data input (image shape: 224 raws and 224x3 columns)
    timesteps = 224        # Timesteps : each raw from an image will feed a deployed timestep cell.
    n_classes = 3          # Number of classes, one class per digit

#display_freq = batch_size    # Frequency of displaying the training results


Dataset type= P7


In [7]:
def reset_graph():
    if 'session' in globals() and session:
        print("\n*** Closing session...")
        session.close()
    tf.reset_default_graph()
    print("\n*** Graph has been reset! Graph should be rebuilt now!\n")

def load_data():
    '''Defaut path to where MNIST dataset is located (after download) :
    ~/.keras/datasets/mnist.npz
    '''
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    #-----------------------------------------------------------
    # Make data separation between valid and train data.
    #-----------------------------------------------------------
    x_valid_num = x_test.shape[0]//2
    part = x_train.shape[0]-x_valid_num

    x_valid = x_train[part:,:]
    y_valid = y_train[part:]
    x_train = x_train[:part,:]
    y_train = y_train[:part]
    
    return x_train, x_valid, x_test, y_train, y_valid, y_test

def load_data_deprecated(mode='train'):
    """
    Function to (download and) load the MNIST data
    :param mode: train or test
    :return: images and the corresponding labels
    """
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)
    if mode == 'train':
        x_train, y_train, x_valid, y_valid = mnist.train.images, mnist.train.labels, \
                                             mnist.validation.images, mnist.validation.labels
        return x_train, y_train, x_valid, y_valid
    elif mode == 'test':
        x_test, y_test = mnist.test.images, mnist.test.labels
    return x_test, y_test


def randomize(x, y):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y.shape[0])
    shuffled_x = x[permutation, :]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y

def get_next_batch(x, y, start, end):
    x_batch = x[start:end]
    y_batch = y[start:end]
    return x_batch, y_batch

# weight and bais wrappers
def weight_variable(shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('Weight',
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('Bias',
                           dtype=tf.float32,
                           initializer=initial)

def RNN(x, weights, biases, timesteps, num_hidden, cell_type='RNN', layer_number=2):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    print("RNN() : Original X shape= {} / Timesteps= {}".format(x.shape, timesteps))
    x = tf.unstack(x, timesteps, 1)
    print("RNN() : Unstacked : Number of input tensors= {} / Input size= {} / Timesteps= {}"\
          .format(len(x), x[0].shape, timesteps))
    print("RNN() : Weights shape, Bias shape = {}, {}".format(weights.shape, biases.shape))

    # Define a rnn cell with tensorflow
    if cell_type == 'RNN' :
            type_cell = rnn.BasicRNNCell(num_hidden)
    elif cell_type == 'LSTM' :
        type_cell = tf.nn.rnn_cell.LSTMCell(num_hidden, name='basic_lstm_cell',state_is_tuple=True)
    elif cell_type == 'GRU' :
        type_cell = tf.contrib.rnn.GRUCell(num_hidden)
    elif cell_type == 'SRNN' :
        list_num_hidden = [num_hidden for _ in range(0,layer_number)]
        list_type_cell = [tf.contrib.rnn.BasicLSTMCell(num_units=n) for n in list_num_hidden]
        type_cell = tf.contrib.rnn.MultiRNNCell(list_type_cell)
    elif cell_type == 'SGRU' :
        list_num_hidden = [num_hidden for _ in range(0,layer_number)]
        list_type_cell = [tf.contrib.rnn.GRUCell(num_units=n) for n in list_num_hidden]
        type_cell = tf.contrib.rnn.MultiRNNCell(list_type_cell)
    else : 
        print("\n*** ERROR : cell type= {} NOT YET SUPPORTED!".format(type_cell))
        return None

    # Get LSTM cell output
    # If no initial_state is provided, dtype must be specified
    # If no initial cell state is provided, they will be initialized to zero
    output, current_state = rnn.static_rnn(type_cell, x, dtype=tf.float32)

    print("\n*** RNN() : Weights shape= {} / Tensor Output= {}".format(weights.shape, output[-1]))
    
    # Linear activation, using rnn inner loop last output
    #return tf.matmul(current_state, weights) + biases
    return tf.matmul(output[-1], weights) + biases

In [8]:
#x_train, y_train, x_valid, y_valid = load_data_deprecated()
#x_test, y_test = load_data_deprecated(mode='test')

import p8_util
if True :
    if dataset_type == 'MNIST' :
        dataset_filename = None
        x_train, x_valid, x_test, y_train, y_valid, y_test, n_classes,tuple_dimension \
        = p8_util.load_dataset(dataset_filename, dataset_type=dataset_type)
        print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, n_classes,tuple_dimension)
    else :
        dataset_filename = './data/arr_keras_X_y_train_test.dump'
        x_train, x_valid, y_train, y_valid, n_classes,tuple_dimension \
        = p8_util.load_dataset(dataset_filename, dataset_type=dataset_type)
        number = x_train.shape[0]
        w = x_train.shape[1]
        h = x_train.shape[2]
        c = x_train.shape[3]
        print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape, n_classes,tuple_dimension)
        x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],-1)
        x_valid = x_valid.reshape(x_valid.shape[0],x_valid.shape[1],-1)
        print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape, n_classes,tuple_dimension)
        
        x_train = np.transpose(x_train, [0,2,1])
        x_valid = np.transpose(x_valid, [0,2,1])
        x_test  = x_valid.copy()
        y_test  = y_valid.copy()
                               
        print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape, n_classes,tuple_dimension)
else :
    x_train, y_train, x_valid, y_valid= load_data_deprecated()
    print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3) 3 (224, 224, 3)
(414, 224, 672) (47, 224, 672) (414, 3) (47, 3) 3 (224, 224, 3)
(414, 672, 224) (47, 672, 224) (414, 3) (47, 3) 3 (224, 224, 3)


In [9]:
y_train.shape, n_classes

((414, 3), 3)

In [10]:
#print(y_train[10])

In [11]:
reset_graph()

# Here-under : 
# 3 classes (colums)
# 4 batches (raws)
logits = [[0.1, 0.5, 0.4],
          [0.8, 0.1, 0.1],
          [0.6, 0.3, 0.2],
          [0.3, 0.3, 0.9],
         ]
labels = [[0, 1, 0],
          [1, 0, 0],
          [0, 0, 1],
          [0, 0, 1],
         ]

acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, 1), predictions=tf.argmax(logits,1))
with tf.Session() as  session :
    session.run(tf.local_variables_initializer())
    session.run(tf.global_variables_initializer())
    print(session.run([acc, acc_op]))
    print(session.run([acc]))


*** Graph has been reset! Graph should be rebuilt now!

[0.0, 0.75]
[0.75]


In [12]:
def init_weight_and_bias(num_hidden_units, n_classes, timesteps, num_input):
    # Placeholders for inputs (x) and outputs(y)

    # create weight matrix initialized randomely from N~(0, 0.01)
    W = weight_variable(shape=[num_hidden_units, n_classes])

    # create bias vector initialized as zero
    b = bias_variable(shape=[n_classes])
    
    return W,b



## y_pred = tf.nn.softmax(output_logits)

# Model predictions
##cls_prediction = tf.argmax(output_logits, axis=1, name='predictions')

In [13]:
num_hidden_units

256

#### Tensorflow Direct Acyclic Graph is built

In [14]:
# Decide to compute accuracy thanks to metrics TF API or not.
reset_graph()
# Graph structure is built : 

_is_tf_metrics_accuracy = True
# Variables initialization : Weights, Bias, X and Y.
W, b = init_weight_and_bias(num_hidden_units, n_classes, timesteps, num_input)

X = tf.placeholder(tf.float32, shape=[None, timesteps, num_input], name='X')
Y = tf.placeholder(tf.float32, shape=[None, n_classes], name='Y')

# Get logits from RNN model
output_logits = RNN(X, W, b, timesteps, num_hidden_units, cell_type=CELL_TYPE, layer_number=RNN_LAYER)
    

# Define the loss function, is inserted into graph; loss function uses y,output_logits.
# cross-e,tropy will apply softmax to logit, compute cross-entropy and will perform average between 
# all classes.
with tf.name_scope(CELL_TYPE+'_'+str(RNN_LAYER)+'LAYER'+'_'+str(num_hidden_units)+'units'):
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_logits), name='loss')
    tf.summary.scalar('loss',loss)

    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)
    with tf.name_scope('accuracy'):
        if _is_tf_metrics_accuracy :
            acc, accuracy= tf.metrics.accuracy(predictions=tf.argmax(output_logits,1)\
                                          ,labels=tf.argmax(Y,1),  name='accuracy')    
            #print("Accuracy= {}".format(accuracy))
        else :    
            # y_prediction is a vector of indexes containing largest value in output_logitstimesteps, num_inputtimesteps, num_inputtimesteps, num_inputtimesteps, num_input
            y_prediction = tf.argmax(output_logits, 1)

            # y_true is a vector of indexes containing largest value in y
            y_true = tf.argmax(Y, 1)

            # A new node is inserted into graph : correct_prediction
            # Following will result as an array of boolean values; True if indexes matches, False otherwise.
            correct_prediction = tf.equal(y_prediction, y_true, name='correct_pred')

            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),  name='accuracy')
    tf.summary.scalar('accuracy', accuracy)
    
#----------------------------------------------------------
# All summary nodes are combined into a single operation.
#----------------------------------------------------------
combined_summary_node = tf.summary.merge_all()

#----------------------------------------------------------------------
# Create a file writer with directory when all events from summary 
# operations are dumped to.
# Graph of all operations in session is also provided to be displayed.
#----------------------------------------------------------------------
summary_writer_train = tf.summary.FileWriter(OUTPUT_DIR+'/train', session.graph)
summary_writer_test = tf.summary.FileWriter(OUTPUT_DIR+'/test', session.graph)
    


*** Closing session...

*** Graph has been reset! Graph should be rebuilt now!

RNN() : Original X shape= (?, 224, 672) / Timesteps= 224
RNN() : Unstacked : Number of input tensors= 224 / Input size= (?, 672) / Timesteps= 224
RNN() : Weights shape, Bias shape = (256, 3), (3,)

*** RNN() : Weights shape= (256, 3) / Tensor Output= Tensor("rnn/gru_cell/add_223:0", shape=(?, 256), dtype=float32)


In [15]:
batch_number = x_train.shape[0]//batch_size
batch_valid = 20
freq_display_per_batch=batch_number//5
global_step=0

print("\nCell Type= {}-{}Layers / Hidden units= {} / Batch size= {} / Batchs number= {} / Epochs={}"\
      .format(CELL_TYPE, RNN_LAYER, num_hidden_units, batch_size, batch_number, epoch_number))

init = tf.global_variables_initializer()


with tf.Session() as session:
    session.run(init)
    session.run(tf.local_variables_initializer())
    for epoch in range(0, epoch_number):
        print("Epoch= {} / {}".format(epoch+1, epoch_number))
        batch_start = 0
        x_train, y_train = randomize(x_train, y_train)
        for batch_iter in range(0, batch_number) :
            batch_end   = (batch_iter+1)*batch_size
            x_batch = x_train[batch_start:batch_end]
            y_batch = y_train[batch_start:batch_end]

            # Reshape x_batch to be compliant with X definition shape that is [None, timesteps, num_input]
            x_batch = x_batch.reshape(batch_size, timesteps, num_input)
            feed_dict_train = {X: x_batch, Y: y_batch}

            session.run(optimizer, feed_dict=feed_dict_train)

            if batch_iter%freq_display_per_batch == 0 :
                loss_train,accuracy_train,summary_train \
                = session.run([loss,accuracy, combined_summary_node],feed_dict=feed_dict_train)
                print("Batch= {0} / Loss={1:.2f} / Train Accuracy={2:.001%}"\
                      .format(batch_iter, loss_train,accuracy_train))
            batch_start = batch_end
            global_step +=1

        # Loss and accuracy evaluation over test dataset.
        # After each EPOCH, new data are selected for evaluation.
        x_test, y_test = randomize(x_valid, y_valid)
        feed_dict_test = {X: x_test[:batch_valid].reshape((-1, timesteps, num_input))\
                           , Y: y_test[:batch_valid]}
        loss_valid, acc_valid, summary_test = session.run([loss, accuracy, combined_summary_node]\
                                                     , feed_dict=feed_dict_test)
        
        
        print("*** Epoch= {0}\t Global steps= {4}\t Validation count= {1}\t Loss={2:.2f}\t Accuracy={3:.01%}"\
              .format(epoch+1, batch_valid, loss_valid,acc_valid, global_step))
        #--------------------------------------------------------------------------
        # Summary data is  writen on disk on location fixed while defining 
        # tf.summary.FileWriter
        #--------------------------------------------------------------------------
        summary_writer_train.add_summary(summary_train, global_step)
        summary_writer_test.add_summary(summary_test, global_step)



Cell Type= GRU-1Layers / Hidden units= 256 / Batch size= 23 / Batchs number= 18 / Epochs=100
Epoch= 1 / 100
Batch= 0 / Loss=1.06 / Train Accuracy=47.8%
Batch= 3 / Loss=1.09 / Train Accuracy=45.7%
Batch= 6 / Loss=1.14 / Train Accuracy=40.6%
Batch= 9 / Loss=1.07 / Train Accuracy=44.6%
Batch= 12 / Loss=1.03 / Train Accuracy=45.2%
Batch= 15 / Loss=1.09 / Train Accuracy=45.7%
*** Epoch= 1	 Global steps= 18	 Validation count= 20	 Loss=1.10	 Accuracy=44.9%
Epoch= 2 / 100
Batch= 0 / Loss=1.04 / Train Accuracy=47.5%
Batch= 3 / Loss=1.05 / Train Accuracy=47.1%
Batch= 6 / Loss=1.03 / Train Accuracy=46.7%
Batch= 9 / Loss=1.04 / Train Accuracy=47.2%
Batch= 12 / Loss=0.95 / Train Accuracy=49.1%
Batch= 15 / Loss=1.05 / Train Accuracy=50.0%
*** Epoch= 2	 Global steps= 36	 Validation count= 20	 Loss=0.96	 Accuracy=50.9%
Epoch= 3 / 100
Batch= 0 / Loss=0.91 / Train Accuracy=52.2%
Batch= 3 / Loss=1.06 / Train Accuracy=52.8%
Batch= 6 / Loss=1.03 / Train Accuracy=53.0%
Batch= 9 / Loss=0.95 / Train Accuracy

Batch= 9 / Loss=0.41 / Train Accuracy=70.1%
Batch= 12 / Loss=0.38 / Train Accuracy=70.2%
Batch= 15 / Loss=0.20 / Train Accuracy=70.4%
*** Epoch= 23	 Global steps= 414	 Validation count= 20	 Loss=1.44	 Accuracy=70.2%
Epoch= 24 / 100
Batch= 0 / Loss=0.29 / Train Accuracy=70.3%
Batch= 3 / Loss=0.29 / Train Accuracy=70.4%
Batch= 6 / Loss=0.23 / Train Accuracy=70.6%
Batch= 9 / Loss=0.34 / Train Accuracy=70.7%
Batch= 12 / Loss=0.33 / Train Accuracy=70.8%
Batch= 15 / Loss=0.15 / Train Accuracy=70.9%
*** Epoch= 24	 Global steps= 432	 Validation count= 20	 Loss=1.52	 Accuracy=70.8%
Epoch= 25 / 100
Batch= 0 / Loss=0.34 / Train Accuracy=70.9%
Batch= 3 / Loss=0.27 / Train Accuracy=71.0%
Batch= 6 / Loss=0.39 / Train Accuracy=71.1%
Batch= 9 / Loss=0.19 / Train Accuracy=71.2%
Batch= 12 / Loss=0.54 / Train Accuracy=71.2%
Batch= 15 / Loss=0.32 / Train Accuracy=71.3%
*** Epoch= 25	 Global steps= 450	 Validation count= 20	 Loss=0.93	 Accuracy=71.2%
Epoch= 26 / 100
Batch= 0 / Loss=0.31 / Train Accuracy=71

Batch= 0 / Loss=0.02 / Train Accuracy=80.6%
Batch= 3 / Loss=0.03 / Train Accuracy=80.7%
Batch= 6 / Loss=0.03 / Train Accuracy=80.7%
Batch= 9 / Loss=0.01 / Train Accuracy=80.8%
Batch= 12 / Loss=0.02 / Train Accuracy=80.9%
Batch= 15 / Loss=0.10 / Train Accuracy=80.9%
*** Epoch= 46	 Global steps= 828	 Validation count= 20	 Loss=1.18	 Accuracy=80.8%
Epoch= 47 / 100
Batch= 0 / Loss=0.01 / Train Accuracy=80.9%
Batch= 3 / Loss=0.04 / Train Accuracy=81.0%
Batch= 6 / Loss=0.02 / Train Accuracy=81.0%
Batch= 9 / Loss=0.03 / Train Accuracy=81.1%
Batch= 12 / Loss=0.01 / Train Accuracy=81.1%
Batch= 15 / Loss=0.02 / Train Accuracy=81.2%
*** Epoch= 47	 Global steps= 846	 Validation count= 20	 Loss=1.61	 Accuracy=81.1%
Epoch= 48 / 100
Batch= 0 / Loss=0.02 / Train Accuracy=81.2%
Batch= 3 / Loss=0.02 / Train Accuracy=81.2%
Batch= 6 / Loss=0.02 / Train Accuracy=81.3%
Batch= 9 / Loss=0.05 / Train Accuracy=81.4%
Batch= 12 / Loss=0.51 / Train Accuracy=81.4%
Batch= 15 / Loss=0.06 / Train Accuracy=81.4%
*** Ep

Batch= 12 / Loss=0.00 / Train Accuracy=85.7%
Batch= 15 / Loss=0.00 / Train Accuracy=85.8%
*** Epoch= 68	 Global steps= 1224	 Validation count= 20	 Loss=0.78	 Accuracy=85.8%
Epoch= 69 / 100
Batch= 0 / Loss=0.00 / Train Accuracy=85.8%
Batch= 3 / Loss=0.00 / Train Accuracy=85.8%
Batch= 6 / Loss=0.00 / Train Accuracy=85.8%
Batch= 9 / Loss=0.00 / Train Accuracy=85.9%
Batch= 12 / Loss=0.00 / Train Accuracy=85.9%
Batch= 15 / Loss=0.00 / Train Accuracy=85.9%
*** Epoch= 69	 Global steps= 1242	 Validation count= 20	 Loss=1.08	 Accuracy=85.9%
Epoch= 70 / 100
Batch= 0 / Loss=0.00 / Train Accuracy=85.9%
Batch= 3 / Loss=0.00 / Train Accuracy=86.0%
Batch= 6 / Loss=0.00 / Train Accuracy=86.0%
Batch= 9 / Loss=0.00 / Train Accuracy=86.0%
Batch= 12 / Loss=0.00 / Train Accuracy=86.1%
Batch= 15 / Loss=0.00 / Train Accuracy=86.1%
*** Epoch= 70	 Global steps= 1260	 Validation count= 20	 Loss=1.78	 Accuracy=86.1%
Epoch= 71 / 100
Batch= 0 / Loss=0.00 / Train Accuracy=86.1%
Batch= 3 / Loss=0.00 / Train Accuracy

Batch= 0 / Loss=0.07 / Train Accuracy=88.3%
Batch= 3 / Loss=0.06 / Train Accuracy=88.3%
Batch= 6 / Loss=0.06 / Train Accuracy=88.3%
Batch= 9 / Loss=0.05 / Train Accuracy=88.3%
Batch= 12 / Loss=0.06 / Train Accuracy=88.4%
Batch= 15 / Loss=0.02 / Train Accuracy=88.4%
*** Epoch= 91	 Global steps= 1638	 Validation count= 20	 Loss=0.90	 Accuracy=88.4%
Epoch= 92 / 100
Batch= 0 / Loss=0.02 / Train Accuracy=88.4%
Batch= 3 / Loss=0.01 / Train Accuracy=88.4%
Batch= 6 / Loss=0.02 / Train Accuracy=88.4%
Batch= 9 / Loss=0.01 / Train Accuracy=88.4%
Batch= 12 / Loss=0.00 / Train Accuracy=88.4%
Batch= 15 / Loss=0.00 / Train Accuracy=88.5%
*** Epoch= 92	 Global steps= 1656	 Validation count= 20	 Loss=1.07	 Accuracy=88.5%
Epoch= 93 / 100
Batch= 0 / Loss=0.01 / Train Accuracy=88.5%
Batch= 3 / Loss=0.01 / Train Accuracy=88.5%
Batch= 6 / Loss=0.00 / Train Accuracy=88.5%
Batch= 9 / Loss=0.00 / Train Accuracy=88.5%
Batch= 12 / Loss=0.00 / Train Accuracy=88.5%
Batch= 15 / Loss=0.01 / Train Accuracy=88.6%
*** 

In [16]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [17]:
#-----------------------------------------------------------------------
# Hidden units = 128*4*2
# Cell Type= SRNN-3Layers / Batch size= 23 / Batchs number= 18 / Epochs=20
#-----------------------------------------------------------------------


In [18]:
x_test.shape, y_test.shape
session

In [19]:
import numpy as np
import tensorflow as tf


pred = np.array([[31, 23,  4, 24, 27, 34],
                 [18,  3, 25,  0,  6, 35],
                 [28, 14, 33, 22, 20,  8],
                 [13, 30, 21, 19,  7,  9],
                 [16,  1, 26, 32,  2, 29],
                 [17, 12,  5, 11, 10, 15]])

y = np.array([[31, 23,  4, 24, 27, 34],
              [18,  3, 25,  0,  6, 35],
              [28, 14, 33, 22, 20,  8],
              [13, 30, 21, 19,  7,  9],
              [16,  1, 26, 32,  2, 29],
              [17, 12,  5, 11, 10, 15]])
pred = np.array([ [1,2,3,4,5]
                 ,[6,4,3,2,1]
                ])
pred = np.array([ [1,2,3,4,5]
                ])
y = np.array([ [1,-6,-4,-3,2]
                ])

#----------------------------------------------------------------
# Returns the index of largest value across axis from a tensor.
#----------------------------------------------------------------
print("Pred= {}/ Shape= {}".format(pred, pred.shape))
print("Y= {} / Shape= {}".format(y, y.shape))
argmax1 = tf.argmax(pred, 1)
argmax2 = tf.argmax(y, 1)
with tf.Session() as session:
    res1 = session.run(argmax1)
    res2 = session.run(argmax2)
    res3 = session.run(tf.equal(res1,res2))
    res4 = session.run(tf.reduce_mean(tf.cast(res3, tf.float32)))
#print(res1,res2,res3, res4)
print("Argmax Pred= {}".format(res1))
print("Argmax    Y= {}".format(res2))
print("Array of equals indexes= {}".format(res3))
print("Mean result of equals indexes= {}".format(res4))

#print(pred[res])

Pred= [[1 2 3 4 5]]/ Shape= (1, 5)
Y= [[ 1 -6 -4 -3  2]] / Shape= (1, 5)
Argmax Pred= [4]
Argmax    Y= [4]
Array of equals indexes= [ True]
Mean result of equals indexes= 1.0
